In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from collections import defaultdict
from yellowbrick.contrib.classifier import DecisionViz
from sklearn.decomposition import PCA
from itertools import product
from imblearn.datasets import make_imbalance
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from yellowbrick.datasets import load_credit
from yellowbrick.features import PCA as PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score 
from scikitplot.metrics import plot_precision_recall, plot_roc, plot_cumulative_gain, plot_lift_curve , plot_confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, classification_report
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    ConfusionMatrixDisplay
)
import warnings
warnings.filterwarnings("ignore")

In [20]:
df_train=pd.read_csv("../Dataset_prepared/Prepared_train.csv")
df_test=pd.read_csv("../Dataset_prepared/Prepared_test.csv")

df_train.drop('actor',axis=1,inplace=True)
df_train.drop('filename',axis=1,inplace=True)

df_test.drop('actor',axis=1,inplace=True)
df_test.drop('filename',axis=1,inplace=True)

c = df_train.select_dtypes(include=['object']).columns.tolist()
a =[x for x in c if x!='emotion']

from sklearn import preprocessing
le=preprocessing.LabelEncoder()
c = df_train.select_dtypes(include=['object']).columns.tolist()
for i in a:
    df_train[i]=le.fit_transform(df_train[i])

for i in a:
    df_test[i]=le.fit_transform(df_test[i])
df_train['emotion']=le.fit_transform(df_train['emotion'])
df_test['emotion']=le.fit_transform(df_test['emotion'])    

from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler, OneHotEncoder

scaler = StandardScaler()
numeric_features = [n for n in df_train.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaled_features = scaler.fit_transform(df_train[numeric_features])
df_train[numeric_features] = scaled_features


scaled_features_test = scaler.transform(df_test[numeric_features])
df_test[numeric_features] = scaled_features_test

col=[x for x in df_train.columns if x!="emotion"]
X_train=df_train[col].values
y_train = np.array(df_train["emotion"])

X_test=df_test[col].values
y_test = np.array(df_test["emotion"])


In [21]:
def print_confMatrix(y_test,y_test_pred):
    cm=confusion_matrix(y_test, y_test_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels =le.classes_)
    fig, ax = plt.subplots(figsize=(10,10))
    disp.plot(ax=ax, xticks_rotation='vertical',cmap='Greens')
    plt.show()

def print_rocCurve(clf,X_test,y_test):
    y_tmp=clf.predict_proba(X_test)
    plot_roc(y_test, clf.predict_proba(X_test),figsize=(10,6))
    plt.show()

# GradientBoosting family

In [22]:

from collections import defaultdict
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 
from catboost import CatBoostClassifier

GradientBoosting

In [5]:
clf=GradientBoostingClassifier(n_estimators=100,learning_rate=1.0, max_depth=10,min_samples_split=2,min_samples_leaf=20,random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.74      0.64        96
           1       0.60      0.56      0.58        96
           2       0.29      0.40      0.34        48
           3       0.56      0.31      0.40        96
           4       0.45      0.50      0.47        96
           5       0.45      0.48      0.46        48
           6       0.41      0.35      0.38        96
           7       0.41      0.42      0.41        48

    accuracy                           0.48       624
   macro avg       0.47      0.47      0.46       624
weighted avg       0.49      0.48      0.47       624



HistGradientBoosting

In [6]:
clf=HistGradientBoostingClassifier(learning_rate=1.0, max_depth=10,min_samples_leaf=20,random_state=0)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.05      0.10        96
           1       0.25      0.01      0.02        96
           2       0.00      0.00      0.00        48
           3       0.32      0.07      0.12        96
           4       0.15      0.84      0.26        96
           5       0.14      0.06      0.09        48
           6       0.34      0.14      0.19        96
           7       0.00      0.00      0.00        48

    accuracy                           0.18       624
   macro avg       0.22      0.15      0.10       624
weighted avg       0.26      0.18      0.11       624



XGB

In [32]:
clf = XGBClassifier(objective='multi:softprob', 
                    max_depth = 10,
                    learning_rate = 1.0,
                    gamma = 0.0,
                    reg_lambda = 1,
                    tree_method='exact', # 'approx'
                    #tree_method = "gpu_hist",
                    use_label_encoder=False,
                    min_samples_leaf=20,
                    min_samples_split=2,
                    random_state=42
                   )
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

[20:07:19] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "min_samples_leaf", "min_samples_split" } are not used.

              precision    recall  f1-score   support

           0       0.61      0.83      0.70        96
           1       0.57      0.51      0.54        96
           2       0.45      0.62      0.52        48
           3       0.53      0.32      0.40        96
           4       0.39      0.43      0.41        96
           5       0.36      0.50      0.42        48
           6       0.38      0.26      0.31        96
           7       0.49      0.44      0.46        48

    accuracy                           0.48       624
   macro avg       0.47      0.49      0.47       624
weighted avg       0.48      0.48      0.47       624



LGBM

In [11]:
clf = LGBMClassifier(boosting_type='gbdt',  #'goss', #'dart' # can specify binasry loss function
                     max_depth=10, # no limit
                     num_leaves=31,
                     n_estimators=100,
                     subsample_for_bin=200000,
                     objective='binary',
                     reg_alpha=0.0, #L1 regularization term on weights
                     reg_lambda=0.0, #L2 regularization term on weights
                     random_state=42
                   )
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.82      0.70        96
           1       0.62      0.60      0.61        96
           2       0.52      0.54      0.53        48
           3       0.49      0.31      0.38        96
           4       0.41      0.43      0.42        96
           5       0.41      0.48      0.44        48
           6       0.36      0.31      0.34        96
           7       0.39      0.40      0.39        48

    accuracy                           0.49       624
   macro avg       0.48      0.49      0.48       624
weighted avg       0.48      0.49      0.48       624



CatBoost

In [12]:
clf = CatBoostClassifier() # convert all features in categorical but its the same then 
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

Learning rate set to 0.081599
0:	learn: 2.0430001	total: 210ms	remaining: 3m 29s
1:	learn: 2.0004478	total: 276ms	remaining: 2m 17s
2:	learn: 1.9598452	total: 342ms	remaining: 1m 53s
3:	learn: 1.9246246	total: 409ms	remaining: 1m 41s
4:	learn: 1.8925424	total: 476ms	remaining: 1m 34s
5:	learn: 1.8627236	total: 541ms	remaining: 1m 29s
6:	learn: 1.8377045	total: 608ms	remaining: 1m 26s
7:	learn: 1.8106543	total: 675ms	remaining: 1m 23s
8:	learn: 1.7798500	total: 741ms	remaining: 1m 21s
9:	learn: 1.7573037	total: 806ms	remaining: 1m 19s
10:	learn: 1.7323822	total: 873ms	remaining: 1m 18s
11:	learn: 1.7128737	total: 941ms	remaining: 1m 17s
12:	learn: 1.6916395	total: 1.01s	remaining: 1m 16s
13:	learn: 1.6699127	total: 1.07s	remaining: 1m 15s
14:	learn: 1.6512518	total: 1.14s	remaining: 1m 14s
15:	learn: 1.6299627	total: 1.21s	remaining: 1m 14s
16:	learn: 1.6083630	total: 1.27s	remaining: 1m 13s
17:	learn: 1.5924567	total: 1.34s	remaining: 1m 12s
18:	learn: 1.5708526	total: 1.4s	remaining: 

Per adesso il migliore di questa famiglia è il catBoost, proviamo adesso a fare sampling + dim red. + delete 21 outliers

In [13]:
tmp=[8,249,292,308,309,311,382,554,832,1247,1283,1358,1481,1492,1702,1703,1704,1756,1757,1758,1801]

#Read datasets
df_train_new=pd.read_csv("../Dataset_prepared/Prepared_train.csv")
df_test_new=pd.read_csv("../Dataset_prepared/Prepared_test.csv")

#Drop useless features
df_train_new.drop('actor',axis=1,inplace=True)
df_train_new.drop('filename',axis=1,inplace=True)
df_test_new.drop('actor',axis=1,inplace=True)
df_test_new.drop('filename',axis=1,inplace=True)

#Drop the 21 outliers
df_train_new=df_train_new.drop(tmp)

#LabelEncode categorical features
le=preprocessing.LabelEncoder()
c = df_train_new.select_dtypes(include=['object']).columns.tolist()
for i in c:
    df_train_new[i]=le.fit_transform(df_train_new[i])

for i in c:
    df_test_new[i]=le.fit_transform(df_test_new[i])

#Normalize pure numeric features 
scaler = StandardScaler()
numeric_features = [n for n in df_train_new.columns if n!="emotion" and n!= "vocal_channel" and n!= "emotional_intensity" and n!= "statement" and n!= "repetition" and n!= "sex" and n!="filename" and n!="actor"]
scaled_features = scaler.fit_transform(df_train_new[numeric_features])
df_train_new[numeric_features] = scaled_features
scaled_features_test = scaler.transform(df_test_new[numeric_features])
df_test_new[numeric_features] = scaled_features_test

#Set X_train,y_train = data to fit models. Set X_test,y_test = data to test models.
col=[x for x in df_train_new.columns if x!="emotion"]
X_train_new=df_train_new[col].values
y_train_new = np.array(df_train_new["emotion"])
X_test_new=df_test_new[col].values
y_test_new = np.array(df_test_new["emotion"])

In [14]:
from imblearn.under_sampling import TomekLinks
from collections import Counter
tl = TomekLinks()
X_res_new, y_res_new = tl.fit_resample(X_train_new, y_train_new)
print('Resampled dataset shape %s' % Counter(y_res_new))

from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

sel = RFE(DecisionTreeClassifier(max_depth=10, min_samples_split=2, min_samples_leaf=20), n_features_to_select=22)
X_train_sel = sel.fit_transform(X_res_new, y_res_new)
print(f" DT New dimension: {X_train_sel.shape}")
X_test_sel = sel.transform(X_test_new)

Resampled dataset shape Counter({0: 254, 1: 249, 4: 249, 6: 248, 3: 245, 5: 139, 2: 128, 7: 127})
 DT New dimension: (1639, 22)


In [15]:
clf = CatBoostClassifier() # convert all features in categorical but its the same then 
clf.fit(X_train_sel, y_res_new)
y_pred = clf.predict(X_test_sel)
print(classification_report(y_test, y_pred))

Learning rate set to 0.081146
0:	learn: 2.0220580	total: 4.31ms	remaining: 4.3s
1:	learn: 1.9803882	total: 12ms	remaining: 5.99s
2:	learn: 1.9367904	total: 19.6ms	remaining: 6.52s
3:	learn: 1.8920044	total: 27.3ms	remaining: 6.81s
4:	learn: 1.8548930	total: 35ms	remaining: 6.97s
5:	learn: 1.8208532	total: 42.6ms	remaining: 7.07s
6:	learn: 1.7830730	total: 50.4ms	remaining: 7.16s
7:	learn: 1.7508970	total: 57.8ms	remaining: 7.17s
8:	learn: 1.7220626	total: 65.7ms	remaining: 7.24s
9:	learn: 1.6983073	total: 73.4ms	remaining: 7.27s
10:	learn: 1.6750137	total: 81.2ms	remaining: 7.3s
11:	learn: 1.6516777	total: 89ms	remaining: 7.32s
12:	learn: 1.6265183	total: 96.6ms	remaining: 7.33s
13:	learn: 1.6039459	total: 104ms	remaining: 7.34s
14:	learn: 1.5881022	total: 112ms	remaining: 7.35s
15:	learn: 1.5691362	total: 120ms	remaining: 7.36s
16:	learn: 1.5493574	total: 127ms	remaining: 7.35s
17:	learn: 1.5289358	total: 135ms	remaining: 7.35s
18:	learn: 1.5122182	total: 142ms	remaining: 7.35s
19:	le

Proviamo grid search per XGB

In [33]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

parameters = {
            'max_depth': [3, 4, 5, 10],
            'learning_rate': [0.1, 0.2, 0.3],
            'gamma': [0, 0.1, 0.2],
            'reg_alpha': [0, 1e-2, 1, 1e1],
            'reg_lambda': [0, 1e-2, 1, 1e1],
}

random_state = np.random.randint(0, 1000)
xgb_model = XGBClassifier(objective='multi:softprob',tree_method='exact', random_state = random_state, n_estimators=50,use_label_encoder=False)
clf = GridSearchCV(xgb_model, parameters, scoring = 'f1_macro', cv = 3)

clf.fit(X_train, y_train)
best_parameters=clf.best_params_
best_model=clf.best_estimator_
print(best_parameters)


{'gamma': 0, 'learning_rate': 0.3, 'max_depth': 3, 'reg_alpha': 0, 'reg_lambda': 10.0}


In [34]:
y_pred=best_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.58      0.86      0.69        96
           1       0.63      0.61      0.62        96
           2       0.47      0.60      0.53        48
           3       0.54      0.23      0.32        96
           4       0.42      0.44      0.43        96
           5       0.47      0.54      0.50        48
           6       0.38      0.33      0.36        96
           7       0.41      0.38      0.39        48

    accuracy                           0.50       624
   macro avg       0.49      0.50      0.48       624
weighted avg       0.49      0.50      0.48       624



In [35]:
parameters = {
            'max_depth': [3, 4, 5, 10],
            'learning_rate': [0.1, 0.2, 0.3],
            'gamma': [0, 0.1, 0.2],
            'reg_alpha': [0, 1e-2, 1, 1e1],
            'reg_lambda': [0, 1e-2, 1, 1e1],
}

random_state = np.random.randint(0, 1000)
xgb_model = XGBClassifier(objective='multi:softprob',tree_method='exact', random_state = random_state, n_estimators=100,use_label_encoder=False)
clf = GridSearchCV(xgb_model, parameters, scoring = 'f1_macro', cv = 3)

clf.fit(X_train, y_train)
best_parameters=clf.best_params_
best_model=clf.best_estimator_
print(best_parameters)


{'gamma': 0, 'learning_rate': 0.3, 'max_depth': 5, 'reg_alpha': 0.01, 'reg_lambda': 1}


In [39]:
y_pred=best_model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.59      0.85      0.69        96
           1       0.60      0.54      0.57        96
           2       0.47      0.58      0.52        48
           3       0.51      0.25      0.34        96
           4       0.42      0.43      0.42        96
           5       0.47      0.54      0.50        48
           6       0.34      0.32      0.33        96
           7       0.43      0.44      0.43        48

    accuracy                           0.49       624
   macro avg       0.48      0.49      0.48       624
weighted avg       0.48      0.49      0.48       624



In [ ]:
parameters = {
            'max_depth': [3, 4, 5, 10],
            'learning_rate': [0.1, 0.2, 0.3],
            'gamma': [0, 0.1, 0.2],
            'reg_alpha': [0, 1e-2, 1, 1e1],
            'reg_lambda': [0, 1e-2, 1, 1e1],
}

random_state = np.random.randint(0, 1000)
xgb_model = XGBClassifier(objective='multi:softprob',tree_method='exact', random_state = random_state, n_estimators=150,use_label_encoder=False)
clf = GridSearchCV(xgb_model, parameters, scoring = 'f1_macro', cv = 3)

clf.fit(X_train, y_train)
best_parameters=clf.best_params_
best_model=clf.best_estimator_
print(best_parameters)

Proviamo grid search per LGBM